# 排序算法

## 问题定义

**Input** 一个包含$n$个数字的序列 $s=<a_1, a_2, a_3, ..., a_n>$.

**Output**原序列$s$的其中一个排列(permutation)$s'=<a_1', a_2', a_3', ..., a_n'>$, 使得$a_1' \leq a_2' \leq ... \leq a_n'$.

## Heap Sort 堆排序

时间复杂度: $O(n log n)$

借助了数据结构 "堆 (Heap)", 所以在将堆排序之前我们需要先了解**堆**的一些基本性质.

### 堆

堆(二叉堆 binary heap) 是一种用**近似**完全二叉树来表示的数据结构 (*完全二叉树的意思为除了最底层, 其余层所有节点都有左右两个子节点*).

而堆也可以用数组来存储, 其中堆的根节点为Heap[1]. 

![堆的表示](http://users.cecs.anu.edu.au/~Alistair.Rendell/Teaching/apac_comp3600/module2/images/Heaps_HeapStructure.png)

对于堆中的某个节点$i$, 计算它的父节点和左右节点可以使用以下方法:

$parent(i)=floor({i \over 2})$

$left(i)=2i$

$right(i)=2i+1$

堆分为两种: **最大堆**和**最小堆**. 对于最大堆来说, 它的特性为:

任意节点$i \neq 1$, 有$Heap[parent(i)] \geq Heap[i]$. 所以最大堆中最大元素存放在根节点. 

最小堆与最大堆相反, 它的特性为:

任意节点$i \neq 1$, 有$Heap[parent(i)] \leq Heap[i]$. 所以最小堆中最小元素存放在根节点. 



In [1]:
def left(i):
    return i * 2

def right(i):
    return i * 2 + 1

def parent(i):
    return int(i / 2)

对于堆排序, 我们使用最大堆. 由二叉树的定义易知堆的$height=\Theta(log n)$

下面定义堆的几种操作:

### Max-heapify (Sink)

该操作用于维护最大堆的性质, 它的输入是一个数组$A$和一个下标$i$. 当它被调用时, 假设以$i$节点的两个子节点为根的两棵子树已经满足最大堆的性质, 而节点$i$的值A\[i\]有可能小于它子节点的值, 该情况不满足最大堆的定义. 所以在max-heapify操作中, A\[i\]会往下**下沉**, 完成操作后以$i$为根节点的子树将满足最大堆的性质. 我们将这个操作起名为下沉 (sink).

**举例** sink(a, 2)的过程模拟
![过程](https://ethanstorage.oss-cn-hangzhou.aliyuncs.com/heapify.png?Expires=1526442200&OSSAccessKeyId=LTAIDJBobPyq7D2H&Signature=k3ptFBzxg71GH7AtoOFVrnoT%2Fc8%3D)

In [8]:
def sink(array, i, heap_size):
    l, r = left(i), right(i)
    largest = i
    if l <= heap_size and array[l] > array[largest]:
        largest = l
    if r <= heap_size and array[r] > array[largest]:
        largest = r
    if largest == i:
        return 
    array[i], array[largest] = array[largest], array[i]
    sink(array, largest, heap_size)

array = [0, 16, 4, 10, 14, 7, 9, 3, 2, 8, 1]
sink(array, 2, len(array) - 1)
print(array)

[0, 16, 14, 10, 8, 7, 9, 3, 2, 4, 1]


#### Sink操作的时间复杂度分析

1. 在判断节点$i$与其两个子节点的关系并且交换位置的步骤复杂度为$\Theta(1)$
2. 在递归调用某一个子节点时, 假设以$i$为根节点的树共有$n$个节点, 那么以其子节点为根的子树至多有$2n\over 3$个节点
3. 只有一次递归调用

综上, 我们可以写出Sink的$T(n)$如下:

$T(n) \leq T({2n \over 3}) + O(1)$

运用Master Theorem, 得出a=1, b=${3 \over 2}=1.5$, d=0. 

$$
T(n)=
\begin{cases}
O(n^dlogn)& ,\text{if } a=b^d\\
O(n^d)& ,\text{if } a < b^d\\
O(n^{log_ba})& ,\text{if } a > b^d
\end{cases}
$$

$b^d=1 = a$, 属于case 1, 所以复杂度为$O(logn)$

### Build heap

建堆的过程非常容易, 只需反复调用Sink操作即可实现. 通过分析我们知道假设数组$A$长度为$n$, 那么$A[n/2+1...n]$都是叶节点, 即它们都是长度为1的堆, 我们只需要关心前半部分也同样满足堆的性质即可.

In [10]:
def build_heap(array):
    heap_size = len(array) - 1
    for i in range(int(heap_size / 2), 0, -1):
        sink(array, i, heap_size)

array = [0, 6, 4, 10, 14, 7, 9, 3, 2, 8, 1]
build_heap(array)
print(array)

[0, 14, 8, 10, 6, 7, 9, 3, 2, 4, 1]


***思考*** 如何证明该操作的正确性?

#### Build heap的时间复杂度分析 \[重点\]

由前文可得到每一次Sink操作消耗$O(log n)$的时间, 并且build-heap进行了${n \over 2}=O(n)$次调用, 所以复杂度为$O(logn) \times O(n) = O(nlogn)$.

然而这是一个很松弛的渐进上界, 因为当对很底层的节点作sink操作时, 虽然堆中的节点数并没有发生改变, 但sink操作的时间会比对根节点进行sink操作短很多. 因此我们可以得出一个更好更紧密的上界. 

通过观察我们知道$n$个节点的堆的高度为$log n$, 且高度正好为$h$的节点数至多有${n \over 2^{h+1}}$个. 而高度为$h$的树, 对其根节点进行sink操作的复杂度为$O(h)$, 所以我们可以将build-heap的复杂度表示为
$$
O\Big(\sum_{h=0}^{logn} {n \over 2^{h+1}} O(h) \Big) = O\Big( n \sum_{h=0}^{logn} {h \over 2^{h+1}} \Big)
$$

令$k=logn, S=\sum_{h=0}^{k} {h \over 2^{h+1}}$, 则$2S=\sum_{h=0}^{k} {h \over 2^{h}}$.

$2S-S=({1\over2}+{2\over4}+{3\over8}+...+{k\over2^k}) - ({1\over4}+{2\over8}+...+{{i-1}\over2^i} + {i\over2^{i+1}})$

$S={1\over2} + {1\over4} + {1\over8} + ... + {1\over2^i} - {i\over2^{i+1}}$

显而易见, 
$$S \lt 1 - {i\over2^{i+1}} \lt 1$$

所以$O\Big( n \sum_{h=0}^{logn} {h \over 2^{h+1}} \Big)=O(n \times S)=O(n)$, 即build-heap的复杂度其实为$ O(n)$, 牛逼得抠脚!

### The MIGHTY heapsort algorithm

接下来就是堆排序的主体部分了. 用自然语言描述堆排序的过程为:

1. 对输入序列$A[1...n]$建堆 (build-heap操作).
2. 由于最大堆的定义是最大的值在根节点, 所以我们将$A[1]$与$A[n]$交换位置, 并将堆的大小-1.
3. 下沉$A[1]$, 因为此时$A[1]$是从最后直接上来的, 并不一定满足堆的性质.
4. 重复2, 3步骤, 直到堆中剩余元素为2.

经过上述过程之后的原序列$A[1...n]$即为排序后的序列, 也就是说, 堆排序是不额外消耗空间的.

图示过程:

![堆排序](https://ethanstorage.oss-cn-hangzhou.aliyuncs.com/heapsort.jpeg?Expires=1526459665&OSSAccessKeyId=LTAIDJBobPyq7D2H&Signature=8R%2BAmCTtuB9IZLsl5gI4hp3somc%3D)

上代码:

In [31]:
def heapsort(array):
    # 在数组前添加一个占位符, 使得真正的数组从1开始
    array.insert(0, 0)
    
    # 建堆
    build_heap(array)
    heap_size = len(array) - 1
    
    # 从当前堆的最后一个叶节点开始与array[1]交换, 并将交换后的节点排除在堆之外
    # 最后下沉array[1]
    for i in range(len(array)-1, 1, -1):
        array[1], array[i] = array[i], array[1]
        heap_size -= 1
        sink(array, 1, heap_size)
    
    # 将最开始的占位符删掉
    array.pop(0)


a = [16, 1, 10, 8, 7, 9, 3, 2, 4, 14]
heapsort(a)
print(a)

[1, 2, 3, 4, 7, 8, 9, 10, 14, 16]


#### 时间复杂度分析

建堆需要$O(n)$的时间, 每次循环内的下沉需要$O(log n)$的时间, 循环n-1次, 所以复杂度为$O(n + n log n)=O(nlogn)$

## 快速排序